# Wissensaggregator Mittelalter und frühe Neuzeit
WIAGweb2

## Daten für Domherren aus dem Digitalen Personenregister einlesen

Übertrage Daten aus der GS-online. Die Daten werden aus der produktiven Datenbank auf eine lokale Datenbank eingelesen: `gso`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [10]:
item_type_id = 6

6

In [11]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

| Datum | Quelle|
|:---|---|
|*2022-02-25*|`gso_in_202202`|
|*2022-05-12*|`gso_in_202205`| 
|*2022-07-26*|`gso_in_202207`| 
|*2022-10-15*|`gso_in_202207`|
|*2022-10-19*|`gso_in_202210`|

In [12]:
gs_db = "gso_in_202210"

"gso_in_202210"

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingeladen.

## Amtsperiode

In [13]:
table_name = gs_db * ".offices";
sql = "SELECT * FROM $(table_name)
WHERE deleted = 0";
df_pr = Wds.sql_df(sql);

In [14]:
Wds.clean_up!(df_pr);

In [15]:
size(df_pr)

(73471, 22)

### Domherren zuordnen

In [16]:
transform!(df_pr, :person_id => ByRow(string) => :id_in_source);

In [17]:
table_name = "item";
sql = "SELECT id AS item_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)";
df_idx = Wds.sql_df(sql);

In [18]:
size(unique(df_idx.item_id))

(9292,)

In [19]:
df_prp = innerjoin(df_pr, df_idx, on = :id_in_source);

In [20]:
length(unique(df_prp.item_id))

9264

In [21]:
size(df_prp), size(unique(df_prp.person_id))

((16449, 24), (9264,))

In [22]:
person_id_mg_oc = setdiff(df_idx.item_id, df_prp.item_id)

28-element Vector{Int32}:
 483406
 488530
 488571
 488572
 488578
 488580
 488581
 488583
 488590
 488591
 488595
 488597
 488598
      ⋮
 488642
 488648
 488716
 489275
 489276
 489277
 489317
 489327
 489334
 489338
 489339
 489346

In [23]:
str_mg = "(" * join(person_id_mg_oc, ", ") * ")"

"(483406, 488530, 488571, 488572, 488578, 488580, 488581, 488583, 488590, 488591, 488595, 488597, 488598, 488628, 488629, 488630, 488642, 488648, 488716, 489275, 489276, 489277, 489317, 489327, 489334, 489338, 489339, 489346)"

*2022-03-10* 14 Domherren sind ohne Amt. Das ist möglich, weil Domherren ergänzt werden, die referenziert werden von der Domherrendatenbank unabhängig von einem Amt  
*2022-04-28* 17 Domherren sind ohne Amt  
*2022-05-12* 20 Domherren sind ohne Amt  
*2022-07-26* 20 Domherren sind ohne Amt  
*2022-10-15* 23 Domherren sind ohne Amt
*2022-10-19* 18 Domherren sind ohne Amt

### Rollen/Ämter zuordnen

In [24]:
table_name = "role"
sql = "SELECT name as role_name, role.id as role_id
FROM $(table_name)"
df_role = Wds.sql_df(sql);

In [25]:
df_prp = leftjoin(df_prp, df_role, on = :bezeichnung => :role_name, matchmissing = :notequal);

In [26]:
df_mg_role_id = subset(df_prp, :role_id => ByRow(ismissing));

In [27]:
size(df_prp, 1), size(df_mg_role_id, 1)

(16449, 1272)

*2022-01-20* Es gibt noch kein Mapping von Ämtern auf normierte Ämter. Die Suche der Webanwendung berücksichtigt aber auch die ursprünglichen Bezeichnungen (`person_role.role_name`).

### Bistümer zuordnen

In [28]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

In [29]:
df_prp = leftjoin(df_prp, df_dioc, on = :dioezese => :diocese_name, matchmissing = :notequal);

### Instutionen zuordnen
Klöster, Domstifte

`klosterid` entspricht `institution.id_gsn`

In [30]:
sql = "SELECT id AS institution_id, name as institution_name, id_gsn FROM institution"
df_inst = Wds.sql_df(sql);

In [31]:
names(df_inst)

3-element Vector{String}:
 "institution_id"
 "institution_name"
 "id_gsn"

In [32]:
df_prp[100:105, [:bezeichnung, :klosterid, :institution]]

,bezeichnung,klosterid,institution
,String?,String?,String?
1,Domherr,832,Domstift Hildesheim
2,Domherr,832,Domstift Hildesheim
3,Domherr,832,Domstift Hildesheim
4,Domherr,832,Domstift Hildesheim
5,Domherr,832,Domstift Hildesheim
6,Domherr,832,Domstift Hildesheim


In [33]:
int_or_missing(x) = ismissing(x) ? missing : parse(Int, x);
transform!(df_prp, :klosterid => ByRow(int_or_missing) => :klosterid);

In [34]:
df_prp = leftjoin(df_prp, df_inst, on = :klosterid => :id_gsn, matchmissing = :notequal);

In [35]:
df_prp[100:105, [:bezeichnung, :klosterid, :institution, :institution_id]]

,bezeichnung,klosterid,institution,institution_id
,String?,Int64?,String?,Int32?
1,Domherr,2066,Domstift Schwerin,80300
2,Domherr,832,Domstift Hildesheim,80025
3,Domvikar,803,Domstift Osnabrück,79983
4,Domvikar,628,Domstift Münster,79825
5,Kanoniker,175,Stift St. Nikolaus,79532
6,Kanoniker,175,Stift St. Nikolaus,79532


### Numerische Daten eintragen

In [36]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
transform!(df_prp, :von => ByRow(num_date_begin) => :num_date_begin);

In [37]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper);
transform!(df_prp, :bis => ByRow(num_date_end) => :num_date_end);

In [38]:
transform!(df_prp, :bis => ByRow(Wds.parse_year_sort) => :date_sort_key);

In [39]:
function update_col(from, to)
    cand = Wds.parse_year_sort(from)
    if ismissing(to) || cand < to
        return cand
    else
        return to
    end
end

update_col (generic function with 1 method)

In [40]:
transform!(df_prp, [:von, :date_sort_key] => ByRow(update_col) => :date_sort_key);

In [41]:
df_prp[560:567, [:person_id, :bezeichnung, :von, :num_date_begin, :bis, :num_date_end, :date_sort_key]]

,person_id,bezeichnung,von,num_date_begin,bis,num_date_end,date_sort_key
,Int32,String?,String?,Int64?,String?,Int64?,Int64
1,282802,Domherr,vor 1721,1671,missing,missing,1721100
2,282820,Kanoniker,vor 1797,1747,missing,missing,1797100
3,282820,Kanoniker,vor 1797,1747,missing,missing,1797100
4,228508,Kanoniker,1458,1458,1469,1469,1458150
5,282823,Domvikar,1387,1387,1421,1421,1387150
6,282823,Domherr,1417,1417,1421,1421,1417150
7,282827,Kanoniker,1473,1473,missing,missing,1473150
8,282827,Domvikar,1464,1464,missing,missing,1464150


In [42]:
count(!ismissing, df_prp.num_date_begin)

15934

Orte zuordnen. Dann sind Daten redundant in der Datenbank, das erleichtert aber die Suche.  
*2022-02-28* Verfolge die Idee zunächst nicht weiter, sondern frage die Orte in der Anwendung ab.

Einträge für diesen Item_Typ löschen

In [43]:
item_type_id

6

In [44]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 6)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  

In [45]:
columns = [
    :dioezese => :diocese_name,
    :von => :date_begin,
    :bis => :date_end,
    :anmerkung => :note,
    :bezeichnung => :role_name,
    :item_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :institution_name => :institution_name,    
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key
];

In [46]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: Rows inserted: 16449
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


16449

## Zeitspanne der Person

In [47]:
table_name = "item"
sql = "SELECT id, num_date_birth, num_date_death
FROM person WHERE id in (SELECT id FROM $(table_name) WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [48]:
size(df_date)

(9292, 3)

Zeitspanne der Ämter

In [49]:
df_pr_begin_0 = dropmissing(df_prp, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :item_id, :num_date_begin), :item_id)

,item_id,num_date_begin
,Int32,Int64
1,480260,1410
,item_id,num_date_begin
,Int32,Int64
1,489551,1436
2,489551,1435


In [50]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [51]:
size(df_pr_date_min)

(9092, 2)

In [52]:
df_pr_date_min[3015:3021, :]

,item_id,date_min
,Int32,Int64
1,483398,1699
2,483399,1467
3,483400,1520
4,483401,1615
5,483402,1751
6,483403,1308
7,483404,1301


`date_max`

In [53]:
df_pr_end_value = dropmissing(df_prp, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :item_id, :num_date_end), :item_id)

,item_id,num_date_end
,Int32,Int64
1,480260,1411
,item_id,num_date_end
,Int32,Int64
1,489551,1452
2,489551,1452


In [54]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [55]:
size(df_pr_date_max)

(5940, 2)

In [56]:
df_pr_date_max[3015:3021, :]

,item_id,date_max
,Int32,Int64
1,485507,1811
2,485508,1552
3,485509,1505
4,485511,1727
5,485512,1820
6,485513,1367
7,485514,1414


In [57]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :item_id);

In [58]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :item_id);

In [59]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [60]:
size(df_date), count(!ismissing, df_date.date_min), count(!ismissing, df_date.date_max)

((9292, 5), 9092, 5940)

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [61]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [62]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [63]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [64]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [65]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [66]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [67]:
dropmissing!(df_date, :date_min);

In [68]:
size(df_date)

(9115, 5)

Trage die geänderten Daten ein

In [69]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [70]:
df_date[206:210, :]

,id,num_date_birth,num_date_death,date_min,date_max
,Int32,Int32?,Int32?,Signed,Signed?
1,480694,missing,missing,1503,1537
2,480695,missing,missing,1196,1200
3,480699,missing,missing,1351,1371
4,480700,missing,missing,968,1005
5,480701,missing,1498,1448,1498


In [71]:
count(ismissing, df_date.date_min), count(ismissing, df_date.date_max), count(ismissing, df_date.id)

(0, 0, 0)

In [72]:
extrema(df_date.id)

(480260, 489551)

In [73]:
describe(df_date)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Signed,Float64,Signed,Int64,Type
1,id,4.84944e5,480260,484973.0,489551,0,Int32
2,num_date_birth,1639.68,925,1679.0,1845,7922,"Union{Missing, Int32}"
3,num_date_death,1543.21,794,1553.0,1932,5082,"Union{Missing, Int32}"
4,date_min,1459.96,732,1464.0,1882,0,Signed
5,date_max,1478.87,768,1485.0,1932,0,"Union{Missing, Signed}"


In [74]:
stmt = DBInterface.prepare(Wds.dbwiag, "UPDATE person SET date_min = ?, date_max = ? WHERE id = ?");

In [75]:
n_loop = 1
for row in eachrow(df_date)
    date_min, date_max, id = Int64.(Tuple(row[[:date_min, :date_max, :id]]))
    DBInterface.execute(stmt, (date_min, date_max, id))
    n_loop += 1
end
n_loop

9116

In [76]:
DBInterface.close!(stmt)

Alternativen: Update über eine eigene Tabelle

In [71]:
sql = "DROP TABLE IF EXISTS date_update"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DROP TABLE IF EXISTS date_update", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [72]:
sql = "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [73]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [74]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [75]:
df_date[1:10, :]

,id,num_date_birth,num_date_death,date_min,date_max
,Int32,Int32?,Int32?,Signed,Signed?
1,301939,missing,missing,1410,1411
2,301940,missing,missing,1543,1588
3,301950,missing,missing,1208,1209
4,301951,missing,missing,1233,1284
5,301952,missing,1341,1324,1341
6,301954,missing,missing,1469,1472
7,301958,missing,missing,1522,1535
8,301960,missing,1527,1464,1527
9,301961,missing,1397,1373,1397


*2022-02-20* nicht zu identifizierender Fehler: UndefRefError: access to undefined reference mit `Wds.filltable!`

Alternative, direkte Schleife

In [76]:
table_name = "date_update";
step_msg = 2000;
for (i, row) in enumerate(eachrow(df_date))
    sql = "INSERT INTO $(table_name) " *
    "VALUES ($(row[:id]), $(row[:date_min]), $(row[:date_max]))"
    DBInterface.execute(Wds.dbwiag, sql)
    if i % step_msg == 0
        @info "Datensatz" i
    end
end

┌ Info: Datensatz
│   i = 2000
└ @ Main In[76]:8
┌ Info: Datensatz
│   i = 4000
└ @ Main In[76]:8
┌ Info: Datensatz
│   i = 6000
└ @ Main In[76]:8
┌ Info: Datensatz
│   i = 8000
└ @ Main In[76]:8


In [77]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 8786, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12